Link to GDrive

In [99]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# **ATTENTION** (cf ci-dessous)

Choisir entre

*   Bloc 1 : Génération d'un fichier csv "aléatoire" à partir du csv initial (479Mo)
*   Bloc 2 : On repart du dernier csv généré pour (par exemple) réexecuter le data cleaning

original (sans doublons)

In [100]:
import pandas as pd
import random

file_path = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/"
file_name = "enron_unique"
file_name_out = "enron_output_500"
file = file_path+file_name+'.csv'
 #
# CHOISIR entre Bloc 1 et Bloc 2 selon que l'on veut regénérer (Bloc 1) ou pas (Bloc 2) un fichier csv initial
# Enlever ou Mettre en commentaires selon le choix
# Bloc 1
#
#n = sum(1 for line in open(file)) - 1 #number of records in file (excludes header)
#s = 500 #desired sample size
#file_name_out = file_name_out+'_output_'+str(s)
#skip = sorted(random.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
#df_emails = pd.read_csv(file, skiprows=skip)

# Bloc 2
#
file_name = "enron_output_500"
file_name_out = file_name+'_bis'
file = file_path+file_name+'.csv'
df_emails = pd.read_csv(file)

print('Fichier de départ : ',file)

df_emails['cc'] = ''                        # pour identfication des personnes en copie
df_emails['NER_header'] = ''                # Spacy NER
df_emails['NER_body'] = ''                  # Spacy NER
df_emails['body_clean'] = ''                # body sans caracteres parasites V1
df_emails['body_principal'] = ''            # body sans caracteres parasites V2
df_emails['body_dict'] = ''                 # body tokenisé
df_emails['summary_TFIDF'] = ''             # extractive summary by TF-IDF
df_emails['summary_TFIDF_sim'] = ''         # calcul de similarité par rapport à body_clean
df_emails['summary_spacy'] = ''             # extractive summary by Spacy
df_emails['summary_spacy_sim'] = ''         # calcul de similarité par rapport à body_clean
df_emails['summary_BART'] = ''              # abstractive summary by HugginFace/BART
df_emails['summary_BART_sim'] = ''          # calcul de similarité par rapport à body_clean
df_emails['summary_T5'] = ''                # abstractive summary by HugginFace/T5
df_emails['summary_T5_sim'] = ''            # calcul de similarité par rapport à body_clean
df_emails['best_sim'] = ''                  # meilleure taux de similarité par rapport à body_clean

df_emails.tail(10)
deb = 0
fin = df_emails.shape[0]
print(deb,fin)

print(file_name_out)


Fichier de départ :  /content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_output_500.csv
0 500
enron_output_500_bis


Fonction - Sauvegarde vers CSV

In [101]:
# SAUVEGARDE en csv du résultat.
def svgd_csv_xls (file_path,file_name):

# to .CSV
  file_extension = ".csv"
  file = file_path+file_name+file_extension
  print("File : ",file)
  df_emails.to_csv(file, encoding='utf-8', index=False)
# df_emails[deb:fin].to_csv(file, encoding='utf-8', index=False)
  print("Svgde effectuée")
  return


# Data Cleaning

In [102]:
# création du champ body_clean
df_emails['body_clean'] = ''                # body sans caracteres parasites


1. DataCleaning



> 1a) Regex



In [103]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import re
import string

def data_clean_1 (tokens):
# Suppression des caractères spéciaux ou inutiles \"',]
  r = re.compile(r"[\\\''\'\"\[\]]+")
  tok=r.sub('', tokens)
  r = re.compile(r"([\(\)]{1,}|,{2,})")
  tok2=r.sub(' ', tok)
  r = re.compile(r"((\.\s?,)+|(, ,)|(^,{1,}))")
  tok3=r.sub('', tok2)
  r = re.compile(r"\s{2,}")
  text=r.sub(' ', tok3)
  return text


# Rajout 08/08
def data_clean_2 (tokens):
  text = tokens.lower()
#  text = re.sub('\[.*?\]', '', text)  # texte entre []
  text = re.sub('https?://\S+|www\.\S+', '', text)  # url
  text = re.sub('<.*?>+', '', text) # ponctuation
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # ponctuation
  text = re.sub('\n', '', text)
#  text = re.sub('\w*\d\w*', '', text)  # chiffres encadrés par caractères

  return text


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [104]:
df_emails.head()

,date,from,to,header,body,cc,NER_header,NER_body,body_clean,body_dict,summary_TFIDF,summary_TFIDF_sim,summary_spacy,summary_spacy_sim,summary_BART,summary_BART_sim,summary_T5,summary_T5_sim,best_sim,body_principal
0,2000-06-27 05:33:00-07:00,phillip.allen@enron.com,kenneth.shulklapper@enron.com,gas storage model,"['', '---------------------- Forwarded by Phil...",,,,,,,,,,,,,,,
1,2001-08-30 11:56:58-07:00,k..allen@enron.com,mike.grigsby@enron.com,FW: Nine Energy Services,"['', '', '', ' -----Original Message-----', 'F...",,,,,,,,,,,,,,,
2,2001-05-13 07:27:00-07:00,john.arnold@enron.com,kimberly.hillis@enron.com,NaN,"['', 'Kim:', 'I will not be at the may 18 mana...",,,,,,,,,,,,,,,
3,2001-09-26 16:28:52-07:00,jennifer.white@oceanenergy.com,john.arnold@enron.com,FW: RaceCarClub 2001 Party - Friday October 5 ...,"['X-FileName: JARNOLD (Non-Privileged).pst', '...",,,,,,,,,,,,,,,
4,2001-10-17 04:57:28-07:00,caroline.abramo@enron.com,john.arnold@enron.com,RE: lot size eol/renaissance/other funds,"['', 'thanks smart ass.. let me know when this...",,,,,,,,,,,,,,,


In [106]:
def sel_email_principal(df):
# on isole le mail INITIAL sans tenir compte des forwarded ou réponses ultérieures
    import re
    result = re.finditer(r'\'-+ Forwarded by.{,700}Subject:', df)
    # print all match object
    deb=0
    fin=0
    texte=""
    liste=[]
    for match_obj in result:
        liste.append(match_obj.span())
    if len(liste)==0:
        return df
    if len(liste)==1:
        deb=int(liste[0][0])
        fin=int(liste[0][1])
        return df[fin:]
    if len(liste)>1:
        deb=int(liste[1][0])
        fin=int(liste[0][1])
        
        return df[fin:deb]
    
# création d'un colonne email uniquement avec l'email principal
df_emails['body_principal']=df_emails['body'].apply(lambda d:sel_email_principal(d))

In [108]:
def sel_email_original(df):
# on isole le mail INITIAL sans tenir compte des forwarded ou réponses ultérieures
    import re
    result = re.finditer(r'\'-+ Original Message.{,700}Subject:', df)
    # print all match object
    deb=0
    fin=0
    texte=""
    liste=[]
    for match_obj in result:
        liste.append(match_obj.span())
    if len(liste)==0:
        return df
    if len(liste)==1:
        deb=int(liste[0][0])
        fin=int(liste[0][1])
        return df[fin:]
    if len(liste)>1:
        deb=int(liste[1][0])
        fin=int(liste[0][1])
        
        return df[fin:deb]
    
# création d'un colonne email uniquement avec l'email principal
df_emails['body_principal']=df_emails['body_principal'].apply(lambda d:sel_email_original(d))

In [92]:
# -----------------------------------------------------------------------------
print("DataCleaning 1/3 en cours pour ",fin, "records.")
df_emails['body_clean'] = df_emails['body'].apply(lambda x : data_clean_1(x))
print("DataCleaning 1/3 terminé pour ",fin, "records.")
df_emails[['body','body_clean']].head(20)

DataCleaning 1/3 en cours pour  500 records.
DataCleaning 1/3 terminé pour  500 records.


,body,body_clean
0,"['', '---------------------- Forwarded by Phil...",---------------------- Forwarded by Phillip K...
1,"['', '', '', ' -----Original Message-----', 'F...",", -----Original Message-----, From: tBradford..."
2,"['', 'Kim:', 'I will not be at the may 18 mana...","Kim:, I will not be at the may 18 management ..."
3,"['X-FileName: JARNOLD (Non-Privileged).pst', '...",X-FileName: JARNOLD Non-Privileged .pst Hope y...
4,"['', 'thanks smart ass.. let me know when this...",thanks smart ass.. let me know when this chan...
5,"['', 'Attached please find the Natural Gas mar...",Attached please find the Natural Gas market a...
6,"['X-Origin: Badeer-R', 'X-FileName: rbadeer.ns...","X-Origin: Badeer-R, X-FileName: rbadeer.nsf Ke..."
7,"['', ""Consistent with FERC's order dated Augus...","Consistent with FERCs order dated August 23, ..."
8,"['', '', '--------- Inline attachment follows ...",--------- Inline attachment follows ---------...
9,"['X-Origin: Bass-E', 'X-FileName: ebass.nsf', ...","X-Origin: Bass-E, X-FileName: ebass.nsf Rob , ..."


> 1b) Suppression de ce qui precede le "subject" dans le ['body']

In [46]:

# suppression dans le corps du texte de tout ce qui precede la mention 'subject:" y compris le subject présent dans le header.
def data_clean_header_in_body(header,body):
  # Suppression de tout ce qui précède l'objet (inclus) dans le corps du mail
  # header = df_emails['header'][id]
  # body = df_emails['body_clean'][id]
  if header == '':
    return body
  ref = 'Subject: '+str(header).strip()+' '
  result = body.find(ref)
  if result != -1:
#    print('ID:',id)
#    print("header:",header,"\n body:",body)
    result +=len(ref)
    texte = body[result:]
#    print("texte:",texte)  
  else:
    texte = body
  # Traitement des mails avec header vide mais comportement Subject dans le body : on supprime tout ce qui précéde jusqu'à  subject
  ref = 'Subject: '
  result = texte.find(ref)
  if result != -1:
    result +=len(ref)
    texte = body[result:]
  
  return texte
# -----------------------------------------------------------------------------

print("DataCleaning 2/3 en cours pour ",fin, "records.")
for id in range(deb,fin):
#  print('ID:',id)
  try:
    header = df_emails['header'][id]
    body = df_emails['body_clean'][id]
    df_emails['body_clean'][id] = data_clean_header_in_body(header,body)
#    print('ID:',id)
#    print(header,' // ',body)
  except:
    print('erreur retraitement body ID',id)
print("DataCleaning 2/3 terminé pour ",fin, "records.")


DataCleaning 2/3 en cours pour  500 records.
DataCleaning 2/3 terminé pour  500 records.


In [47]:
print("DataCleaning 3/3 en cours pour ",fin, "records.")
df_emails['body_clean'] = df_emails['body_clean'].apply(lambda x : data_clean_2(x))
print("DataCleaning 3/3 terminé pour ",fin, "records.")

DataCleaning 3/3 en cours pour  500 records.
DataCleaning 3/3 terminé pour  500 records.


In [97]:
df_emails['body_principal']=df_emails['body'].apply(lambda d:sel_email_principal(d))
df_emails['body_principal'] = df_emails['body_principal'].apply(lambda x : data_clean_1(x))

In [110]:

svgd_csv_xls (file_path,file_name_out)

exit

File :  /content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_output_500_bis.csv
Svgde effectuée


Sauvegarde au format csv